Coucou

In [1]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from model.track_collection import TrackCollection
from utils.collection_splitter import splitter

In [8]:
# loading track collection
track_collection = TrackCollection()
track_collection.load(os.path.join('data', 'track_collection_y.json'))
user_collections = splitter(track_collection, 100, 0.2)

user_dfs = []
for user_collection in user_collections:
    user_dfs.append(user_collection.to_dataframe())

df_track_collection = track_collection.to_dataframe()

In [9]:
matrix = df_track_collection[['id']]

i = 0
for udf in user_dfs:
    ndf = udf[['id', 'rating_score']]
    matrix = matrix.merge(ndf, on='id', how='left').rename(columns = {'rating_score': 'user_'+str(i)})
    i += 1

matrix = matrix.filter(regex='^user_[0-9]+$').transpose().fillna(0)
matrix

,0,1,2,3,4,5,6,7,8,9,...,3120,3121,3122,3123,3124,3125,3126,3127,3128,3129
user_0,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
user_1,0.0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_2,0.0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_3,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
user_4,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
user_5,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.51,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_6,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.5,0.0
user_7,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
user_8,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.51,...,0,0,0,0,0,0,0,0,0.5,0.0
user_9,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
